## Visualize error in reconstructed parameters
This notebook visualizes the error in the reconstructed parameters after calibration, i.e., the MAP point.

- The true log-diffusion coefficient is determined by the tissue segmentations and applied in the forward simulation with a DG-0 indicator function. The error is computed and projected in a DG-1 finite element space and then rasterized in the NIfTI (data space) for visualization with `matplotlib`
- The true log-reaction coefficient is homogeneous. The comparison is calculated in the native CG-1 finite element space and then rasterized in the NIfTI (data space) for visualization.

In [ ]:
import os
import sys

import ufl
import dolfin as dl
import numpy as np
import nibabel
import matplotlib.pyplot as plt

sys.path.append(os.environ.get("HIPPYLIB_PATH"))
import hippylib as hp

sys.path.append(os.path.join(os.getenv("DT4CO_PATH"), "src"))
from dt4co.synth import synthExperiment
from dt4co.utils.mesh_utils import load_mesh_subs
from dt4co.utils.fenics_io import read_mv_from_h5
from dt4co.utils.model_utils import solveIndicators
from dt4co.utils.data_utils import niftiPointwiseObservationOp, rasterizeFunction, nifti2Function, get_slice_number

In [2]:
MESH_FPATH = "/storage1/transfer/gtp/sub-00101_study/mesh/full64-all.h5"
MAP_FPATH = "/storage1/transfer/gtp/sub-00101_study/rdtx_freq1_bip/rdtx_freq1_param_data.h5"
PATIENT_DIR = "/storage/graham/data/UPENN-GBM/sub-00101"
REF_NII = os.path.join(PATIENT_DIR, "tumor_fs.nii")

SLICES = [65, 80, 95, 110]
FIGSIZE = (10, 10)
light_black = [0.1, 0.1, 0.1]
FONTSIZE = 48
LABEL_FS = 36
FONTWEIGHT = "bold"

OUTPUT_DIR = "error_figs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# Load in the mesh and subdomains.
COMM = dl.MPI.comm_world
mesh, subs, bndrys = load_mesh_subs(COMM, MESH_FPATH)

# Set up the experiment and function spaces.
exp = synthExperiment()
    
#  Set up variational spaces for state and parameter.
Vhtrue = exp.setupFunctionSpaces(mesh, mle=False)
Vh = exp.setupBIPFunctionSpaces(mesh, mle=False)
VhDG0 = dl.FunctionSpace(mesh, "DG", 0)
VhDG1 = dl.FunctionSpace(mesh, "DG", 1)
VhCG1 = dl.FunctionSpace(mesh, "CG", 1)
chi_gm = solveIndicators(mesh, subs, 1)

In [4]:
# Get the true parameter.
mtrue = exp.trueParameter(Vhtrue, sample=False)
mtruefun = hp.vector2Function(mtrue, Vhtrue[hp.PARAMETER])
mtrue_dgm, mtrue_dwm, mtrue_k = ufl.split(mtruefun)

# Read back the MAP point.
mfun = dl.Function(Vh[hp.PARAMETER])
mmap = hp.MultiVector(mfun.vector(), 1)
read_mv_from_h5(COMM, mmap, Vh[hp.PARAMETER], MAP_FPATH, name=["map"])

mapfun = hp.vector2Function(mmap[0], Vh[hp.PARAMETER])
map_d, map_k = mapfun.split()

In [5]:
# Compute error in the MAP estimate for log-diffusion in the DG1 space.
map_dpw = map_d * chi_gm + map_d * (1. - chi_gm)
error_expr = (dl.Constant(np.log(exp.DG_TRUE)) - map_d) * chi_gm + (dl.Constant(np.log(exp.DW_TRUE)) - map_d) * (1. - chi_gm)
error_fn_md = dl.project(error_expr, VhDG1)

# Compute error in the MAP estimate for log-reaction in the CG1 space.
error_expr = (dl.Constant(np.log(exp.K_TRUE)) - map_k)
error_fn_mk = dl.project(error_expr, VhCG1)

# Assemble pointwise observation operators.
BopDG1 = niftiPointwiseObservationOp(REF_NII, VhDG1)
BopCG1 = niftiPointwiseObservationOp(REF_NII, VhCG1)

In [6]:
# Write the output.
rasterizeFunction(error_fn_md, VhDG1, REF_NII, "md_error_fn.nii", BopDG1)
rasterizeFunction(error_fn_mk, VhCG1, REF_NII, "mk_error_fn.nii", BopCG1)

### Display the errors in the reconstructed log-diffusion field

In [7]:
errornii = nibabel.load("md_error_fn.nii")
VABS = 3  # check this for different reconstructions.

for sidx, slice in enumerate(SLICES):
    vv = get_slice_number(errornii, -slice)
    
    fig = plt.figure(figsize=FIGSIZE)
    ax = fig.add_subplot(111)
    ax.imshow(errornii.get_fdata()[:, :, vv], cmap='seismic', vmin=-VABS, vmax=VABS)
    ax.axis('off')
    
    plt.savefig(os.path.join(OUTPUT_DIR, f"error_map_md_{slice}.pdf"), dpi=300)
    
    if sidx == 0:
        cbar = fig.colorbar(ax.images[0], location='top')
        cbar.set_label('log(mm$^3$/day)', fontsize=FONTSIZE, color=light_black, fontfamily='sans-serif', fontweight=FONTWEIGHT)
        cbar.ax.tick_params(labelsize=LABEL_FS, color=light_black)
        
        plt.savefig(os.path.join(OUTPUT_DIR, f"error_map_md_{slice}_cbar.pdf"), dpi=300)
    
    plt.close(fig)

### Display the errors in the reconstructed log-reaction field

In [8]:
errornii = nibabel.load("mk_error_fn.nii")
VABS = 0.8  # check this for different reconstructions.

for sidx, slice in enumerate(SLICES):
    vv = get_slice_number(errornii, -slice)

    fig = plt.figure(figsize=FIGSIZE)
    ax = fig.add_subplot(111)
    ax.imshow(errornii.get_fdata()[:, :, vv], cmap='seismic', vmin=-VABS, vmax=VABS)
    ax.axis('off')
    
    plt.savefig(os.path.join(OUTPUT_DIR, f"error_map_mk_{slice}.pdf"), dpi=300)
    
    if sidx == 0:
        cbar = fig.colorbar(ax.images[0], location='top')
        cbar.set_label('log(1/day)', fontsize=FONTSIZE, color=light_black, fontfamily='sans-serif', fontweight=FONTWEIGHT)
        cbar.ax.tick_params(labelsize=LABEL_FS, color=light_black)
        
        plt.savefig(os.path.join(OUTPUT_DIR, f"error_map_mk_{slice}_cbar.pdf"), dpi=300)
    
    plt.close(fig)

### Cleanup

In [9]:
os.remove("md_error_fn.nii")
os.remove("mk_error_fn.nii")